In [ ]:
# install Pint if necessary
try:
  from pint import UnitRegistry
except ImportError:
  !pip install pint

In [ ]:
# download modsim.py if necessary

from os.path import basename, exists


def download(url):
  filename = basename(url)
  if not exists(filename):
    from urllib.request import urlretrieve
    local, _ = urlretrieve(url, filename)
    print('Downloaded ' + local)


download('https://github.com/AllenDowney/ModSimPy/raw/master/' +
         'modsim.py')

In [ ]:
import pandas as pd
from modsim import *
import matplotlib.pyplot as plt
import numpy as np
import random

## Menghapus kolom yang tidak diperlukan

In [ ]:
data_have_index = [
  "sample/anxiety-disorders-males-vs-females.csv",
  "sample/bipolar-disorder-in-males-vs-females.csv",
  "sample/depression-males-vs-females.csv",
  "sample/eating-disorders-in-males-vs-females.csv",
  "sample/Mental health Depression disorder Data.csv",
  "sample/schizophrenia-in-males-vs-females.csv"
]

for file_path in data_have_index:
  df = pd.read_csv(file_path)
  if 'index' in df.columns:
    df = df.drop('index', axis=1)
  df.to_csv(file_path, index=False)

## Monte Carlo simulation
Penjelasan:

1. Setelah looping menginisialisasikan `min_val` dan `max_val` yang diambil dari hasil menghitung rata2 penyakit mental dari 100 ribu orang pertahunnya.
2. `preval_values` akan memiliki sebuah array yang dimulai dari `min_val` sampai `max_val` dengan panjang `system.num_points`
3. `num_male_with_disease` menghitung jumlah laki laki atau permupan dengan penyakit saat ini. Dengan asumsi populasi dan `preval_value` dipilih secara acak. Nilai akan dikalikan dan dibagi 100.
4. `num_suicide_male` mensimulasikan jumlah kasus bunuuh diri diantara laki2 atau perempuan saat ini. Binomial akan menghasilkan bilangan acak pada kasus bunuh diri berdasarkan distribusi yang diberikan

In [ ]:
def run_simulation(state, system):
  total_suicide_male = 0
  total_suicide_female = 0

  # Simulasi untuk laki-laki
  for disease in system.prevalence_range.keys():
    min_val, max_val = system.prevalence_range[disease]['male']
    preval_values = np.linspace(min_val, max_val, system.num_points)
    num_male_with_disease = int(pop_male * np.random.choice(preval_values) / 100)
    num_suicide_male = np.random.binomial(num_male_with_disease, system.suicide_risk[disease])
    total_suicide_male += num_suicide_male

  # Simulasi untuk perempuan
  for disease in system.prevalence_range.keys():
    min_val, max_val = system.prevalence_range[disease]['female']
    preval_values = np.linspace(min_val, max_val, system.num_points)
    num_female_with_disease = int(pop_female * np.random.choice(preval_values) / 100)
    num_suicide_female = np.random.binomial(num_female_with_disease, system.suicide_risk[disease])
    total_suicide_female += num_suicide_female

  return total_suicide_male, total_suicide_female

## Running simulasi Monte Carlo

In [ ]:
for pop_male, pop_female in state.populations:
  state.results_male = []
  state.results_female = []
  prediction_arr = []
  for _ in range(system.iterations):
    suicide_male, suicide_female = run_simulation(state, system)
    state.results_male.append(suicide_male)
    state.results_female.append(suicide_female)

  prediction_arr.append({
    'pop_male': pop_male,
    'pop_female': pop_female,
    'average_male': np.mean(state.results_male),
    'average_female': np.mean(state.results_female)
  })

  state.predictions.append(prediction_arr)

  # Plotting hasil simulasi
  plt.figure(figsize=(14, 7))
  plt.hist(state.results_male, bins=30, alpha=0.7, label='Male Suicides')
  plt.hist(state.results_female, bins=30, alpha=0.7, label='Female Suicides')
  plt.xlabel('Number of Suicides')
  plt.ylabel('Frequency')
  plt.title(
    f'Monte Carlo Simulation of Suicide Cases Due to Mental Health Issues (Population: {pop_male} males, {pop_female} females)')
  plt.legend()
  plt.show()

  # Summary statistics
  mean_male = np.mean(state.results_male)
  mean_female = np.mean(state.results_female)
  print(f'Average suicides in males: {mean_male}')
  print(f'Average suicides in females: {mean_female}')

## Helper function untuk predict
1. `filter_prediction_data(state, output)`
  Fungsi ini digunakan untuk memfilter data prediksi berdasarkan output yang diinginkan. Output dapat berupa "pop_male", "pop_female", "average_suicide_male", atau "average_suicide_female".

2. `sum_prediction_data(size)`
Fungsi ini digunakan untuk menghitung total dari nilai tertinggi atau terendah dari populasi yang diuji. Ukuran populasi yang diuji dapat berupa "besar" atau "kecil".

3. `mean_suicide(state, key)`
Fungsi ini digunakan untuk menghitung rata-rata nilai dari data prediksi berdasarkan kunci yang diberikan. Kunci dapat berupa "pop_male", "pop_female", "average_suicide_male", atau "average_suicide_female".

4. `calculate_growth_ratios(arr)`
Fungsi ini digunakan untuk menghitung rasio pertumbuhan dari sebuah array data. Ini berguna untuk menganalisis tren pertumbuhan data dari waktu ke waktu.

5. `convert_arr_string_into_integer(arr)`
Fungsi ini digunakan untuk mengonversi setiap elemen dalam daftar dari string menjadi integer. Ini diperlukan ketika data awalnya dalam format string dan perlu diubah menjadi integer untuk analisis numerik.

In [ ]:
# Simulasi prediksi
# Mendapatkan range tertinggi dan terendah dari populasi yang diuji diatas.

def filter_prediction_data(state, output):
  """
    Memfilter data prediksi berdasarkan output yang diinginkan.
    
    Parameters:
        state (object): Objek state yang berisi data prediksi.
        output (str): Output yang diinginkan, seperti "pop_male", "pop_female", 
                      "average_suicide_male", atau "average_suicide_female".
                      
    Returns:
        list: Daftar nilai yang telah difilter berdasarkan output yang diinginkan.
  """

  temp_arr = []
  for prediction_data in state.predictions:
    for prediction_value in prediction_data:
      if output == "pop_male":
        temp_arr.append(prediction_value["pop_male"])
      if output == "pop_female":
        temp_arr.append(prediction_value["pop_female"])
      if output == "average_suicide_male":
        temp_arr.append(prediction_value["average_male"])
      if output == "average_suicide_female":
        temp_arr.append(prediction_value["average_female"])
  return temp_arr


def sum_prediction_data(size):
  """
    Menghitung total dari nilai tertinggi atau terendah dari populasi yang diuji.
    
    Parameters:
        size (str): Ukuran populasi yang diuji, "besar" atau "kecil".
        
    Returns:
        float: Total dari nilai tertinggi atau terendah dari populasi yang diuji.
  """
  return np.sum(np.max(filter_prediction_data(state, "pop_male"))) + np.sum(
    np.max(filter_prediction_data(state, "pop_female"))) if size == "besar" else np.sum(
    np.min(filter_prediction_data(state, "pop_male"))) + np.sum(np.min(filter_prediction_data(state, "pop_female")))


def mean_suicide(state, key):
  """
    Menghitung rata-rata nilai dari data prediksi berdasarkan kunci yang diberikan.
    
    Parameters:
        state (object): Objek state yang berisi data prediksi.
        key (str): Kunci untuk menentukan data yang akan dihitung rata-ratanya.
        
    Returns:
        float: Nilai rata-rata dari data prediksi yang sesuai dengan kunci yang diberikan.
  """
  return np.mean(filter_prediction_data(state, key))


def calculate_growth_ratios(arr):
  """
    Menghitung rasio pertumbuhan dari sebuah array data.
    
    Parameters:
        arr (list): Daftar data yang akan dihitung rasio pertumbuhannya.
        
    Returns:
        list: Daftar rasio pertumbuhan antar elemen dalam array data yang telah diurutkan.
  """
  sorted_arr = sorted(arr)
  print(sorted_arr)
  growth_ratios = []

  for i in range(len(sorted_arr) - 1):
    ratio = sorted_arr[i + 1] / sorted_arr[i]
    growth_ratios.append(ratio)

  return growth_ratios


def convert_arr_string_into_integer(arr):
  """
    Mengonversi setiap elemen dalam daftar dari string menjadi integer.
    
    Parameters:
        arr (list): Daftar string yang akan dikonversi menjadi integer.
        
    Returns:
        list: Daftar integer hasil konversi.
  """

  return [int(x) for x in arr]

In [ ]:
sum_prediction_data("kecil")

## Simulasi predict

In [ ]:
def run_prediction_simulate(num_loop, populations, state):
  num_simulation = num_loop[1] - num_loop[0]

  suicide_male = []
  suicide_female = []

  average_suicide_male = np.linspace(np.min(filter_prediction_data(state, "average_suicide_male")),
                                     np.max(filter_prediction_data(state, "average_suicide_male")), 33)
  average_suicide_female = np.linspace(np.min(filter_prediction_data(state, "average_suicide_female")),
                                       np.max(filter_prediction_data(state, "average_suicide_female")), 33)

  if num_simulation >= 1000:
    if (sum_prediction_data("kecil") <= np.min(populations)) or (
      sum_prediction_data("besar") >= np.max(populations)):

      for _ in range(num_simulation):
        if flip(0.5):
          temp_rand_male = random.randint(int(np.min(average_suicide_male)),
                                          int(np.max(average_suicide_male)))
          temp_rand_female = random.randint(int(np.min(average_suicide_female)),
                                            int(np.max(average_suicide_female)))
          suicide_male.append(temp_rand_male)
          suicide_female.append(temp_rand_female)
        else:
          if len(suicide_male) > 0:
            suicide_male.pop(random.randint(0, len(suicide_male) - 1))
          if len(suicide_female) > 0:
            suicide_female.pop(random.randint(0, len(suicide_female) - 1))

  return suicide_male, suicide_female